# File to analyze graphics of default simulation

## Imports to collect data and plot graphics

In [1]:
from components.graphic_generator import Graphic_Gen
from components.data_manager import Data_Manager
import components.network_data as nwd
from components.utils.enums import Colors
import numpy as np
import pandas as pd

## Constants

In [2]:
PATH_TO_CSV: str = "data/topology_simulation/topology_simulation.csv"
NUMBER_OF_NODES: tuple = tuple([12*i for i in range(1, 108, 12)])
PARAMS_TUPLE: tuple[float, float, float] = (0.1, 0.3, 0.5)
COLORS_LIST: tuple[Colors, Colors, Colors] = (Colors.YELLOW, Colors.ORANGE, Colors.RED)

## Load csv

In [3]:
dm: Data_Manager = Data_Manager()
loaded: bool = dm.load_csv(PATH_TO_CSV)
if not loaded:
    print("File don't loaded")

## Convert csv to pandas DataFrame

In [ ]:
dframe: pd.DataFrame = dm.csv_to_dataframe()